In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Classroom/[Genap 20 21] - IN276 - Kelas A Pencarian Informasi Media Online, Selasa pkl. 15.00 - 17.30/UTS/question_philoit.csv')

df = df[['created_at','content']]
df['created_at']= pd.to_datetime(df['created_at'])
df['content'] = df['content'].apply(str)

Mengambil data question philoIT pada rentang bulan Februari 2020 hingga bulan April 2020 

In [5]:
df = df[(df['created_at'] > '2020-02-01') & (df['created_at'] < '2020-04-30')]

df.head()

,created_at,content
1788,2020-02-01 00:37:00,Apa yang seharusnya kita kerjakan saat liburan?
1789,2020-02-01 00:41:34,Apa makanan khas daerah kalian ?
1790,2020-02-01 00:41:34,Apa makanan khas daerah kalian ?
1791,2020-02-01 00:41:34,Apa makanan khas daerah kalian ?
1792,2020-02-01 01:25:41,Menurut kamu yang akan juara club mana ditahun...


# Stopwords, Stemming, and Tokenizing

Mengambil stemming bahasa Indonesia dari module Sastrawi

In [6]:
!pip install Sastrawi

     |████████████████████████████████| 215kB 4.0MB/s 


In [7]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

Mengambil stopwords bahasa Indonesia dari module NLTK

In [8]:
# load nltk's Indonesian stopwords as variable called 'stopwords'
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
nltk.download('punkt')
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Melakukan TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0.005, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(df['content']) #fit the vectorizer to content

print(tfidf_matrix.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arti', 'asa', 'buah', 'bum', 'enak', 'hadap', 'ikan', 'ira', 'jenak', 'kali', 'kena', 'kerja', 'khusus', 'laku', 'langsung', 'lihat', 'maksud', 'masuk', 'mena', 'nyata', 'olah', 'orang', 'rupa', 'sampa', 'sangkut', 'sekal', 'sekira', 'sin', 'tuju', 'upa', 'utama'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(3138, 133)


In [11]:
# get the first vector out (for the first document) 
first_vector_tfidf_matrix=tfidf_matrix[0] 
 
# place tf-idf values in a pandas data frame 
df0 = pd.DataFrame(first_vector_tfidf_matrix.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df0.sort_values(by=["tfidf"],ascending=False)


,tfidf
kerja,1.0
air,0.0
paham,0.0
nyata,0.0
of,0.0
...,...
faktor,0.0
enak,0.0
ekonomi,0.0
efektif,0.0


In [12]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df0

,tfidf
air,0.0
ajar,0.0
alam,0.0
alami,0.0
alas,0.0
ambil,0.0
anak,0.0
aplikasi,0.0
arti,0.0
asa,0.0


# Mengekstrak banyak frekuensi keyword

Pada keyword yang terdapat dalam tabel tfidf, saya akan memilih keyword "indonesia"
Kemudian akan dicari frekuensi kemunculkan kata indonesia pada setiap dokumen

In [13]:
def count_indo(list_word):
  return list_word.count('indonesia')

In [15]:
frekuensi_kata = df
frekuensi_kata['indonesia'] = frekuensi_kata['content'].map(count_indo)
freq_kata = frekuensi_kata[['created_at','indonesia']]
freq_kata.head()

,created_at,indonesia
1788,2020-02-01 00:37:00,0
1789,2020-02-01 00:41:34,0
1790,2020-02-01 00:41:34,0
1791,2020-02-01 00:41:34,0
1792,2020-02-01 01:25:41,0


In [18]:
# Menyimpan freq kata ke dalam menjadi csv
freq_kata.to_csv('/content/drive/MyDrive/Classroom/[Genap 20 21] - IN276 - Kelas A Pencarian Informasi Media Online, Selasa pkl. 15.00 - 17.30/UTS/freq_kata.csv')